In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aligo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def cleanStemPost(post):
    urls = re.compile(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})')
    numbers = re.compile(r'\d+(\.\d+)?')
    posessivePronouns = re.compile(r"’s")
    apostrophe=re.compile(r"’")
    someSigns =re.compile(r"\\n|\\r")
    punctuation = re.compile(r"[^\w\s]")
    whitespaces = re.compile(r'\s+')
    leadTrailWhitespace = re.compile(r'^\s+|\s+?$')
    
    cleanPost = post.lower()
    cleanPost = urls.sub('url',cleanPost)
    cleanPost = numbers.sub('nmbr',cleanPost)
    cleanPost = posessivePronouns.sub('',cleanPost)
    cleanPost = apostrophe.sub('',cleanPost)
    cleanPost = someSigns.sub('',cleanPost)
    cleanPost = punctuation.sub(' ',cleanPost)
    cleanPost = whitespaces.sub(' ',cleanPost)
    cleanPost = leadTrailWhitespace.sub('',cleanPost)
    
    ps = nltk.PorterStemmer()
    
    # Create an empty list containing Stemmed words
    stemmed_list = []
    
    post_words = cleanPost.split(" ")
    
    # Iterate through every word to Stem
    for word in post_words:
        stemmed_list.append(ps.stem(word))
        
    # Join the list
    stemmed_post = " ".join(stemmed_list)
    
    cleanPost = stemmed_post
    
    # stop words
    with open('./../stop_words_no_punct.data', 'rb') as filehandle:
        # read the data as binary data stream
        stop_words_no_punct = pickle.load(filehandle)
        
    post_without_stop_words = []

    text_tokens = word_tokenize(cleanPost)
    tokens_without_stop_words = [word for word in text_tokens if not word in stop_words_no_punct]
    post_without_stop_words = (" ").join(tokens_without_stop_words)

    cleanPost = post_without_stop_words
    
    return cleanPost

In [3]:
testPost = "Ive always had a few mutuals at a time growing up, but never a proper close friend. Let alone that one person who feels that mutual “you are my first choice” for. Until the start of last year :) I saw her almost every single day and we knew everything about each other. We were both each others best friend and even my anxiety filled, obsessive, overthinking brain could know that for sure. I dont know what happened. I dont know what i did wrong. She started hanging out with this group of “pretty popular kids” who are really racist and big bullies. Slowly she started changing, and i could tell she just didnt want to hang out with me anymore. As painful as it was i let it happen and didnt ask to come over anymore or text her unnecessarily because the last thing i want to do is annoy her. Its hard not knowing what i did wrong and i miss being close to her. I just want her to like me again."
parsedPost = cleanStemPost(testPost)
print(parsedPost)

ive alway mutual time grow never proper close friend let alon one person feel mutual first choic start last year saw almost everi singl day knew everyth best friend even anxieti fill obsess overthink brain could know sure know happen know wrong start hang thi group pretti popular kid realli racist big bulli slowli start chang could tell want hang anymor pain wa let happen ask come anymor text unnecessarili becaus last thing want annoy hard know wrong miss close want like


In [4]:
def find_features(post):
    words = word_tokenize(post)
    features = {}
    
    with open('word_features.data', 'rb') as filehandle:
        # read the data as binary data stream
        word_features = pickle.load(filehandle)
    
    for word in word_features:
        features[word] = (word in words)
    return features

In [5]:
def transformSubreddit(x):
    switcher={
            0:'depression',
            1:'unpopularopinion',
            2:'lonely',
            3:'MachineLearning',
         }
    return switcher.get(x)

In [6]:
def transformDepression(x):
    switcher={
            0:'r/depression',
            1:'not r/depression'
         }
    return switcher.get(x)

In [7]:
def classifySubreddit(post):
    modelFile = './Models/finalized_model_subreddits.sav'
    post = cleanStemPost(post)
    
    loaded_model = pickle.load(open(modelFile, 'rb'))
    
    featureset = find_features(post)
    
    result = loaded_model.classify(featureset)
        
    return transformSubreddit(result)

In [8]:
def classifyDepression(post):
    modelFile = './Models/finalized_model_is_depression.sav'
    post = cleanStemPost(post)
    
    loaded_model = pickle.load(open(modelFile, 'rb'))
    
    featureset = find_features(post)
    
    result = loaded_model.classify(featureset)
        
    return transformDepression(result)

### Example

In [9]:
result = classifySubreddit(testPost)
print(result)

lonely


In [10]:
result = classifyDepression(testPost)
print(result)

not r/depression
